In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
%load_ext nb_black
%matplotlib inline

<IPython.core.display.Javascript object>

In [3]:
secop = catalog.load("secop_2_cont_clean")

[08/13/22 18:34:18] INFO     Loading data from 'secop_2_cont_clean' (ParquetDataSet)...         ]8;id=611983;file:///opt/homebrew/Caskroom/miniforge/base/envs/dm_project/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=987936;file:///opt/homebrew/Caskroom/miniforge/base/envs/dm_project/lib/python3.10/site-packages/kedro/io/data_catalog.py#343\343]8;;\

<IPython.core.display.Javascript object>

In [4]:
secop

,index,nombre_entidad,departamento,ciudad,orden,proceso_de_compra,referencia_del_contrato,estado_contrato,descripcion_del_proceso,modalidad_de_contratacion,...,Financiero,Inmobiliaria,Profesionales,AdministracionPublica,Artisticas,ValorAgregado,Impuestos,PIB,Poblacion,days_ref
0,0,corporacion autonoma regional de santander cas,santander,san gil,nacional,CO1.BDOS.1762573,0O115-2021,terminado,prestar los servicios de apoyo a la gestión a ...,contratacion directa,...,0.228486,0.712503,0.378373,0.969653,0.137737,7.756796,1.611073,9.367856,79020.0,7716
1,1,corporacion autonoma regional de santander cas,santander,san gil,nacional,CO1.BDOS.2089644,00851-2021,terminado,prestar los servicios profesionales como ingen...,contratacion directa,...,0.228486,0.712503,0.378373,0.969653,0.137737,7.756796,1.611073,9.367856,79020.0,7864
2,2,corporacion autonoma regional de santander cas,santander,san gil,nacional,CO1.BDOS.2082062,00835-2021,terminado,prestar los servicios como ingeniero agricola ...,contratacion directa,...,0.228486,0.712503,0.378373,0.969653,0.137737,7.756796,1.611073,9.367856,79020.0,7859
3,3,corporacion autonoma regional de santander cas,santander,san gil,nacional,CO1.BDOS.2226116,1064-2021,terminado,prestar los servicios profesionales como admin...,contratacion directa,...,0.228486,0.712503,0.378373,0.969653,0.137737,7.756796,1.611073,9.367856,79020.0,7922
4,4,corporacion autonoma regional de santander cas,santander,san gil,nacional,CO1.BDOS.2225429,1053-2021,terminado,prestar los servicios profesionales como ingen...,contratacion directa,...,0.228486,0.712503,0.378373,0.969653,0.137737,7.756796,1.611073,9.367856,79020.0,7919
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1654284,1654284,esm comando aereo de combate n°5,antioquia,rionegro,nacional,CO1.BDOS.2998235,011-00-DIGSA-JEFSA-CACOM-5-ESM-2022,en ejecucion,prestacion de servicios profesionales y de apo...,contratacion directa,...,0.197383,0.338584,0.350351,0.464716,0.106617,3.688967,0.365551,4.054518,435195.0,8216
1654285,1654285,esm comando aereo de combate n°5,antioquia,rionegro,nacional,CO1.BDOS.2685266,001-02-DIGSA-JEFSA-CACOM5- ESM3029-2022,en ejecucion,contratar 03 auxiliares de enfermería como pre...,contratacion directa,...,0.197383,0.338584,0.350351,0.464716,0.106617,3.688967,0.365551,4.054518,435195.0,8061
1654286,1654286,personeria municipal de santa rosalia vichada,vichada,santa rosalia,territorial,CO1.BDOS.2752225,CO1.PCCNTR.3566501,en ejecucion,adquisición de seguro póliza multiriesgos que ...,menor cuantia,...,0.000136,0.000320,0.000046,0.002903,0.000076,0.007830,0.000233,0.008063,961055.0,8080
1654287,1654287,personeria municipal de santa rosalia vichada,vichada,santa rosalia,territorial,CO1.BDOS.2577561,CO1.PCCNTR.3296171,en ejecucion,prestación de servicios profesionales para la ...,contratacion directa,...,0.000136,0.000320,0.000046,0.002903,0.000076,0.007830,0.000233,0.008063,961055.0,8061


<IPython.core.display.Javascript object>